In [ ]:
import array_api_compat.torch as xp
import torch
import parallelproj
from array_api_compat import to_device
import array_api_compat.numpy as np
import matplotlib.pyplot as plt
import time

dict_data = np.load("slices_8_lors_40.npy", allow_pickle=True).item()

xstart = xp.asarray(dict_data['xstart']).cuda()
xend = xp.asarray(dict_data['xend']).cuda()
img_origin = xp.asarray(dict_data['img_origin']).cuda()
voxel_size = xp.asarray(dict_data['voxel_size']).cuda()
img_shape = dict_data['img_shape']
x_gt = xp.asarray(dict_data['x']).cuda()
meas = xp.asarray(dict_data['meas']).cuda() + 100

fwd = lambda x: parallelproj.joseph3d_fwd(xstart, xend, x, img_origin, voxel_size)

back = lambda y: parallelproj.joseph3d_back(xstart, xend, img_shape, img_origin, voxel_size, y)


start = time.time()
for i in range(10):
    tmp = fwd(x_gt)
    torch.cuda.synchronize(x_gt.device)
end_t = time.time()


start = time.time()
for i in range(10):
    tmp = back(meas)
    torch.cuda.synchronize(x_gt.device)
end_t = time.time()

sens_img = back(xp.ones_like(meas))

x = xp.ones_like(sens_img)

start = time.time()
for i in range(10000):
    em_precond = x/sens_img
    ratio = meas/(fwd(x)+100.)
    torch.cuda.synchronize(x_gt.device)
    bp_ratio = back(ratio)
    torch.cuda.synchronize(x_gt.device)
    x = em_precond*bp_ratio
end_t = time.time()

xp.save(x.cpu(), "parallel_proj_recon.torch")

import matplotlib.pyplot as plt
from ipywidgets import interact
def show_slices(slice_idx):
    plt.imshow(x[:,slice_idx,:].cpu(), cmap='gray')
    plt.colorbar()
    plt.show()

interact(show_slices, slice_idx=(0, x.shape[1]-1, 1))